## Link to article

This notebook is included in the documentation, where the interactive Plotly charts show up. See:
https://pegasystems.github.io/pega-datascientist-tools/Python/articles/vf_analysis.html

In [ ]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import plotly.io as pio

pio.renderers.default = "notebook_connected"

# Value Finder Analysis
Every Value Finder simulation populates a dataset, the **pyValueFinder** dataset.  This dataset contains a lot more information than is what is currently presented on screen.

The data held in this dataset can be analysed to uncover insights into your decision framework. This notebook provides a sample analysis of the Value Finder simulation results.

In the data folder we've stored a copy of such a dataset, generated from an (internal) demo application (CDHSample). To run this notebook on your own data, you should export the **pyValueFinder** dataset from Dev Studio then follow the instructions below.

This is how Value Finder results of the sample data are presented in Pega (8.6, it may look different in other versions):

![Pega value finder screen](pegarun_8_6_0.png)

For the sample provided, the relevant action setting is 1.2%. There are 10.000 customers, 3491 without actions, 555 with only irrelevant actions and 5954 with at least one relevant action.

PDSTools defines a class **ValueFinder** that wraps the operations on this dataset. The "datasets" import is used for the example but you won't need this if you load your own Value Finder dataset.

Just like with the **ADMDatamart** class, you can supply your own path and filename as such:
```python
vf = ValueFinder(path = '[PATH TO DATA]', filename="[NAME OF DATASET EXPORT]")
```

- If only a path is supplied, it will automatically look for the latest file. 
- It is also possible to supply a dataframe as the 'df' argument directly, in which case it will use that instead. 


In [ ]:
from pdstools import ValueFinder, datasets
import polars as pl
import numpy as np

# Load sample data - replace with your own data:
# vf = ValueFinder(path='...', filename='...')
vf = datasets.sample_value_finder()

When reading the data, we filter out unnecessary data, and the result is kept in the `df` property:

In [ ]:
vf.df.head(5).collect()

## Basic Analysis: Pie Charts and Customer Segmentation

The piechart shown in platform is based on a propensity threshold. For the sample data, this threshold follows from a propensity quantile of 5.2%.

The `plot.pie_charts()` function shows piecharts for all stages in the engagement policies (in platform you only see the last one). The threshold is calculated automatically, but you can specify it explicitly.

In [ ]:
# Automatic threshold calculation
vf.plot.pie_charts()

# Or specify threshold explicitly (e.g., 5.2% quantile)
vf.plot.pie_charts(quantiles=[0.052])

Hover over the charts to see the details. For the sample data, the rightmost pie chart corresponds to the numbers in Pega as shown in the screenshot above.

* Red = customers not receiving any action
* Yellow = customers not receiving any "relevant" actions, sometimes also called "under served"
* Green = customers that receive at least one "relevant" action, sometimes also called "well served"

With "relevant" defined as having a propensity above the threshold. This defaults to the 5th percentile.

## Propensity Analysis

Insights into the propensity distribution per stage are crucial. The `plot.propensity_threshold()` function visualizes this distribution. You often see a spike at 0.5, which corresponds to models without responses (their propensity defaults to 0.5/1 = 0.5).

The dotted vertical line represents the computed threshold.

In [ ]:
vf.plot.propensity_threshold()

These different propensities represent 

* *pyModelPropensity* = the actual propensities from the models
* *pyPropensity* = model or random propensity, depending on the ModelControl (or, when models are executed from an extension point after the standard Predictions, their propensity, but such a configuration is not supported by Value Finder)
* *FinalPropensity* = the propensity after possible adjustments by Thompson Sampling; Thompson Sampling basically smoothes the propensities, you would expect any peak at 0.5 caused by empty models to be smoothed out

We can also look at the propensity distributions across the different stages. This is based on the model propensities, not any of the subsequent overrides:

In [ ]:
vf.plot.propensity_distribution()

## Threshold Simulation

The effect of the propensity threshold selection on the number of actions for a customer can be simulated by supplying a list of quantiles or propensities. This generates aggregated counts per stage for different threshold values.

In [ ]:
# Simulate thresholds from 1% to 99% in 1% increments
vf.plot.pie_charts(quantiles=np.arange(0.01, 1, 0.01))

The further to the left you put the slider threshold, the more "green" you will see. As you raise the threshold, more customers will be reported as getting "not relevant" actions.

The same effect can be visualized as a funnel. Use `plot.distribution_per_threshold()` to show the threshold on the x-axis.

In [ ]:
# Default threshold range
vf.plot.distribution_per_threshold()

# Custom threshold range (1% to 99% in 1% increments)
vf.plot.distribution_per_threshold(quantiles=np.arange(0.01, 1, 0.01))

## Funnel Analysis

You can analyze how individual actions are distributed across stages. Since there are typically many actions, it's useful to filter to a specific group or issue.

The `plot.funnel_chart()` function shows how actions flow through the stages. **Rule of thumb**: If there are only a few actions in each stage, or if certain actions are completely filtered out between stages, this may indicate strong filtering issues.

In [ ]:
# Filter to Sales issue actions (adjust query for your data)
vf.plot.funnel_chart("Name", query=pl.col("Issue") == "Sales")

### Hierarchical Funnel Views

Start with a coarse-grained view at the Issue level, then drill down to Group or individual Action level as needed.

In [ ]:
# High-level view: Issue level
vf.plot.funnel_chart("Issue")

# Mid-level view: Groups within Sales issue
vf.plot.funnel_chart("Group", query=pl.col("Issue") == "Sales")